## ANODI-Captum Demonstration

In this short demo we will give a very basic explanation on how to use our library with the model interpretability tool [Captum](https://captum.ai/).

We will use the MTAD-GAT-Model and the ECG-200-Datasets for our example, since all algorithms (using PyTorch and FastAI) are handeled analogously.

We start with the usual imports needed to use the algorithm as well as the IntegratedGradients from Captum. For this method we (can) only consider the model (see Captum_demo_feature_ablation for explanation).

In [4]:
import os
import sys

# change this to your systems settings
current_dir = os.path.dirname(os.path.abspath(('/Users/timon/Documents/ANODI/anodi/anodi/tests/captum_demo.ipynb')))
parent_dir = os.path.dirname(current_dir)
sys.path.append(parent_dir)

In [5]:
from algorithm.MTAD_GAT import MTAD_GAT

from data.download import *
from data.DatasetSpecification import DATASETS
from metrics.cmetrics import *
from captum.attr import IntegratedGradients
import torch


Now we train the MTAD-GAT algorithm and obtain the prepared PyTorch model from the FastAI-Learner.

In [25]:
train = DATASETS["ECG200_TRAIN"]
test = DATASETS["ECG200_TEST"]

alg = MTAD_GAT(dataset_specification=train, 
               batch_size=28, 
               window_length=4,
               learning_rate=1e-3,
               L=1,
               verbose=True,
               dynamic_pot=True,
               level=0.6)
alg.fit(epoch_num=1)

model = alg.learner.model

/Users/timon/Documents/ANODI/anodi/anodi/data/download.py:78: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df = pandas.read_csv(


epoch     train_loss  valid_loss  time    


/Users/timon/Documents/ANODI/anodi/anodi/data/preprocessing.py:13: FutureWarning: DataFrame.interpolate with method=bfill is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df = df.interpolate(method="bfill", axis=0)


0         2.941625    None        00:00                                                


/Users/timon/miniforge3/envs/anodi/lib/python3.10/site-packages/fastprogress/fastprogress.py:73: UserWarning: Your generator is empty.
  warn("Your generator is empty.")


Since the model return a quadruple at predict, we only need the test_pred_df as prediciton.
Next, we pass the (modified) model to the IntegratedGradients and prepare test inputs and the baseline inputs.

In [26]:
def modified_f(in_vec):
    res = model.forward(in_vec)
    return res[2][0]


ig = IntegratedGradients(modified_f)



Finally we use the integrated Gradients method to obtain information on which features had which impact on the outputs of the model. Note that the output of the model is a reconstruction of the inputs since we are dealing with an unsupervised learning algorithm.

For demonstration purposes, we use random and zero inputs here.

In [27]:

input = torch.rand(28,4,96)
baseline = torch.zeros(28,4,96)

Use Integrated Gradients to get attributions and delta.

In [28]:
attributions, delta = ig.attribute(input, baseline, return_convergence_delta=True)
print('IG Attributions:', attributions)
print('Convergence Delta:', delta)

IG Attributions: tensor([[[-3.9913e-08, -8.8001e-07, -5.8852e-07,  ...,  6.9981e-07,
           1.4898e-06,  5.5813e-07],
         [-7.3169e-07, -1.7112e-07,  9.5067e-07,  ..., -3.3682e-07,
          -7.3534e-09, -8.1155e-08],
         [ 2.6222e-07, -3.2600e-07, -1.1314e-06,  ..., -3.4570e-07,
          -8.9932e-08, -8.0977e-08],
         [ 4.5227e-07, -1.0824e-07, -8.7441e-08,  ..., -1.2056e-06,
          -8.6982e-07, -1.4850e-07]],

        [[-2.1055e-07, -1.7675e-07,  1.5221e-07,  ...,  1.0617e-06,
           1.8596e-07,  9.3661e-08],
         [ 2.4079e-07, -2.1432e-07,  4.3604e-07,  ..., -3.4992e-07,
           6.3780e-07,  4.1985e-07],
         [-2.4201e-07, -8.7014e-07, -3.7085e-07,  ...,  3.6410e-07,
          -7.3937e-07, -1.1535e-07],
         [ 1.0717e-06, -3.8623e-07,  1.1559e-07,  ..., -2.2903e-07,
          -1.6260e-06, -1.8149e-07]],

        [[ 8.6348e-08, -2.1598e-07,  7.7145e-09,  ...,  5.0976e-08,
           3.6291e-08, -7.4281e-08],
         [-2.0409e-07, -6.9764e-07